In [1]:
import networkx as nx
import sqlite3
import pathSearch, db

# 1. 領域指定
points = [
    [35.20902036505841, 136.86844825744632],
    [35.140265534798395, 137.02620506286624]
]
y1, x1, y2, x2 = pathSearch.rectangleArea(points)

# 2. 道路データ取得（link, length形式の取得）
link, length = db.getRectangleRoadData(y1, x1, y2, x2)

G = pathSearch.linkToGraph(link, length)
pathSearch.connectGraph(G)
sp = pathSearch.ShortestPathFinder(G)

chosen_indices, subset_list = pathSearch.set_cover(points, 0.3, sp)
for idx in chosen_indices:
    print(subset_list[idx])



{'[35.2068686, 136.8688981]', '[35.2075966, 136.8690422]', '[35.2091581, 136.8715567]', '[35.208352, 136.8712862]', '[35.207361, 136.869722]', '[35.207938, 136.871159]', '[35.2071975, 136.8682761]', '[35.2091687, 136.8700098]', '[35.2078445, 136.8697635]', '[35.2087484, 136.8714234]', '[35.2078057, 136.8684029]', '[35.2095485, 136.8687254]', '[35.2093444, 136.8693652]'}
{'[35.140371, 137.0248696]', '[35.1423538, 137.0246496]', '[35.1414107, 137.0238128]', '[35.1407527, 137.0247838]', '[35.1427674, 137.0264922]', '[35.1415949, 137.0261141]', '[35.1415949, 137.0241776]', '[35.1399673, 137.0257787]', '[35.1415773, 137.0246443]', '[35.1412045, 137.0261034]', '[35.1415817, 137.0247355]', '[35.140478, 137.024217]', '[35.1419678, 137.0239308]', '[35.1412045, 137.0247462]', '[35.1419721, 137.0261088]', '[35.1405743, 137.0243348]', '[35.139985, 137.0261195]', '[35.140376, 137.023819]', '[35.1419634, 137.0246496]', '[35.140255, 137.023228]', '[35.1407527, 137.026098]', '[35.1395551, 137.0257815]

In [1]:
shp = ['a', 'b', 'c']
candidates = [['A'], ['B1', 'B2'], ['C1', 'C2', 'C3']]

shp_dash = []
candidates_dash = []

for s, cand in zip(shp, candidates):
    n = len(cand)
    shp_dash.extend([s] * n)
    candidates_dash.extend([cand] * n)  # cand自体をそのまま複数回

print(shp_dash)
# ['a', 'b', 'b', 'c', 'c', 'c']

print(candidates_dash)

['a', 'b', 'b', 'c', 'c', 'c']
[['A'], ['B1', 'B2'], ['B1', 'B2'], ['C1', 'C2', 'C3'], ['C1', 'C2', 'C3'], ['C1', 'C2', 'C3']]
